# Spanning trees

Anton Antonov   
July 2024

----

## Setup

In [1]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Path;
use Graph::Petersen;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;
use Math::DistanceFunctions;
use Math::Nearest;
use Hash::Merge;

use JavaScript::D3;
use WWW::MermaidInk;

In [2]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [3]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

In [4]:
my $title-color = 'Silver';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

{background => 1F1F1F, edge-thickness => 3, title-color => Silver, vertex-size => 6}

-----

## Small Geo-points set

A company is planning a fiber network for a number of Chicago suburbs. It only has the right of way for its fiber along certain corridors. Some of those corridors might be more expensive. Find the subgraph of connection corridors that connect every suburb with the lowest total cost:

In [5]:
my @dsEdges =
        %('from'=>'Maywood','to'=>'Downers','weight'=>14),
        %('from'=>'Maywood','to'=>'Oakbrook','weight'=>28),
        %('from'=>'Maywood','to'=>'Addison','weight'=>10),
        %('from'=>'Downers','to'=>'Grove','weight'=>25),
        %('from'=>'Downers','to'=>'Stickney','weight'=>23),
        %('from'=>'Downers','to'=>'Wheaton','weight'=>20),
        %('from'=>'Downers','to'=>'Oakbrook','weight'=>27),
        %('from'=>'Oakbrook','to'=>'Stickney','weight'=>11),
        %('from'=>'Oakbrook','to'=>'Addison','weight'=>17),
        %('from'=>'Addison','to'=>'McCook','weight'=>21),
        %('from'=>'Grove','to'=>'Wheaton','weight'=>10),
        %('from'=>'Stickney','to'=>'Wheaton','weight'=>15),
        %('from'=>'Stickney','to'=>'Bellwood','weight'=>20),
        %('from'=>'Wheaton','to'=>'Bellwood','weight'=>10),
        %('from'=>'Bellwood','to'=>'McCook','weight'=>28);

deduce-type(@dsEdges)

Vector(Struct([from, to, weight], [Str, Str, Int]), 15)

In [6]:
my $g = Graph.new(@dsEdges)

Graph(vertexes => 9, edges => 15, directed => False)

In [7]:
#% js
$g.edges(:dataset)
==> js-d3-graph-plot(
    width => 800,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => { strength => -800}, collision => {radius => 50}}
    )

In [8]:
my $stree = $g.find-spanning-tree;

Graph(vertexes => 9, edges => 8, directed => False)

In [9]:
#% js
$g.edges
==> js-d3-graph-plot(
    highlight => $stree.edges,
    width => 800,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => { strength => -800}, collision => {radius => 50}}
    )

-------

## Grid graph

In [10]:
my $g = Graph::Grid.new(n => 4, m => 6)

Graph(vertexes => 24, edges => 38, directed => False)

In [11]:
my $stree = $g.find-spanning-tree

Graph(vertexes => 24, edges => 23, directed => False)

In [12]:
#% js
$g.edges
==> js-d3-graph-plot(
    highlight => [|$stree.vertex-list, |$stree.edges],
    width => 500,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => {strength => -160, iterations => 2}, collision => {radius => 4, iterations => 1}, link => {distance => 0}}
    )

-----

## Florida cities

Build an interstate highway system joining the cities of the state of Florida, USA.

Here is a dataset of large enough cities:

In [13]:
my @cities = city-data().grep({ $_<State> eq 'Florida' && $_<Population> ≥ 50_000 });
@cities.elems

78

In [14]:
#% html
@cities.head(4) ==> to-html

State,Elevation,Longitude,City,Country,Population,Latitude,ID,LocationLink
Florida,5,-81.6613021,Jacksonville,United States,949611,30.3370193,United_States.Florida.Jacksonville,"http://maps.google.com/maps?q=30.3370193,-81.6613021&z=12&t=h"
Florida,2,-80.2086152,Miami,United States,442241,25.775163,United_States.Florida.Miami,"http://maps.google.com/maps?q=25.775163,-80.2086152&z=12&t=h"
Florida,1,-82.4796734,Tampa,United States,384959,27.9700861,United_States.Florida.Tampa,"http://maps.google.com/maps?q=27.9700861,-82.4796734&z=12&t=h"
Florida,30,-81.375789,Orlando,United States,307573,28.533513,United_States.Florida.Orlando,"http://maps.google.com/maps?q=28.533513,-81.375789&z=12&t=h"


Here is the corresponding weighted graph:

In [15]:
my %coords = @cities.map({ $_<City> => ($_<Latitude>, $_<Longitude>) });
my @dsEdges = (%coords.keys X %coords.keys ).map({ %(from => $_.head, to => $_.tail, weight => geo-distance(|%coords{$_.head}, |%coords{$_.tail} )) })

[{from => Poinciana, to => Poinciana, weight => 0} {from => Poinciana, to => Kendall, weight => 294608.235321117} {from => Poinciana, to => Brandon, weight => 83106.88748326803} {from => Poinciana, to => Boca Raton, weight => 237132.94663520565} {from => Poinciana, to => North Miami, weight => 278999.2724277294} {from => Poinciana, to => Weston, weight => 248732.07536148472} {from => Poinciana, to => Lauderhill, weight => 250915.7156946892} {from => Poinciana, to => Sanford, weight => 77275.197816045} {from => Poinciana, to => Coconut Creek, weight => 241586.6061205076} {from => Poinciana, to => Kendale Lakes, weight => 288654.53294163366} {from => Poinciana, to => Port Saint Lucie, weight => 142671.12679341412} {from => Poinciana, to => Pembroke Pines, weight => 259343.74657906033} {from => Poinciana, to => Homestead, weight => 312500.4304670341} {from => Poinciana, to => Miami Gardens, weight => 270836.8914958427} {from => Poinciana, to => North Port, weight => 137723.5839993298} {fr

In [16]:
my $g = Graph.new(@dsEdges, :!directed)

Graph(vertexes => 78, edges => 3081, directed => False)

In [17]:
my $stree = $g.find-spanning-tree(method => 'kruskal')

Graph(vertexes => 78, edges => 77, directed => False)

In [18]:
#my $stree2 = $g.find-spanning-tree(method => 'prim')

()

In [19]:
%coords.nodemap({.reverse}).Hash

{Alafaya => (-81.1857451 28.5283727), Apopka => (-81.5309734 28.7037531), Boca Raton => (-80.1062924 26.3728553), Bonita Springs => (-81.7869289 26.357498), Boynton Beach => (-80.0807345 26.5283266), Bradenton => (-82.5748617 27.4909209), Brandon => (-82.2992552 27.9359851), Cape Coral => (-81.9973306 26.6431192), Carol City => (-80.26992 25.942121), Clearwater => (-82.7662648 27.9794747), Coconut Creek => (-80.1846979 26.2790785), Coral Springs => (-80.2592915 26.2707826), Davie => (-80.2850284 26.0791425), Daytona Beach => (-81.0881451 29.1973282), Deerfield Beach => (-80.1254222 26.3117134), Delray Beach => (-80.0904783 26.4568423), Deltona => (-81.2109776 28.9051205), Doral => (-80.3579892 25.8151765), Fort Lauderdale => (-80.143896 26.141305), Fort Myers => (-81.8319561 26.61901), Fountainbleau => (-80.3494448 25.7743113), Gainesville => (-82.345878 29.6788314), Hialeah => (-80.302865 25.8699408), Hollywood => (-80.1645743 26.0311054), Homestead => (-80.4457729 25.4673459), Horizo

In [20]:
#% js
$stree.edges
==> js-d3-graph-plot(
    vertex-coordinates => %coords.nodemap(*.reverse)».List.Hash,
    width => 700,
    height => 1000,
    :$background, :$title-color, :$edge-thickness, 
    vertex-size => 4,
    vertex-label-font-size => 9,
    margins => {right => 200}
    )

--------

## Africa routes

In [21]:
my @africaEdges = %('from'=>'Algeria','to'=>'Libya','weight'=>1),%('from'=>'Algeria','to'=>'Mali','weight'=>1),%('from'=>'Algeria','to'=>'Mauritania','weight'=>1),%('from'=>'Algeria','to'=>'Morocco','weight'=>1),%('from'=>'Algeria','to'=>'Niger','weight'=>1),%('from'=>'Algeria','to'=>'Tunisia','weight'=>1),%('from'=>'Algeria','to'=>'WesternSahara','weight'=>1),%('from'=>'Libya','to'=>'Niger','weight'=>1),%('from'=>'Libya','to'=>'Tunisia','weight'=>1),%('from'=>'Libya','to'=>'Chad','weight'=>1),%('from'=>'Libya','to'=>'Egypt','weight'=>1),%('from'=>'Libya','to'=>'Sudan','weight'=>1),%('from'=>'Mali','to'=>'Mauritania','weight'=>1),%('from'=>'Mali','to'=>'Niger','weight'=>1),%('from'=>'Mali','to'=>'BurkinaFaso','weight'=>1),%('from'=>'Mali','to'=>'Guinea','weight'=>1),%('from'=>'Mali','to'=>'IvoryCoast','weight'=>1),%('from'=>'Mali','to'=>'Senegal','weight'=>1),%('from'=>'Mauritania','to'=>'WesternSahara','weight'=>1),%('from'=>'Mauritania','to'=>'Senegal','weight'=>1),%('from'=>'Morocco','to'=>'WesternSahara','weight'=>1),%('from'=>'Niger','to'=>'Chad','weight'=>1),%('from'=>'Niger','to'=>'BurkinaFaso','weight'=>1),%('from'=>'Niger','to'=>'Benin','weight'=>1),%('from'=>'Niger','to'=>'Nigeria','weight'=>1),%('from'=>'Chad','to'=>'Sudan','weight'=>1),%('from'=>'Chad','to'=>'Nigeria','weight'=>1),%('from'=>'Chad','to'=>'Cameroon','weight'=>1),%('from'=>'Chad','to'=>'CentralAfricanRepublic','weight'=>1),%('from'=>'Egypt','to'=>'Sudan','weight'=>1),%('from'=>'Sudan','to'=>'CentralAfricanRepublic','weight'=>1),%('from'=>'Sudan','to'=>'Eritrea','weight'=>1),%('from'=>'Sudan','to'=>'Ethiopia','weight'=>1),%('from'=>'Sudan','to'=>'SouthSudan','weight'=>1),%('from'=>'BurkinaFaso','to'=>'IvoryCoast','weight'=>1),%('from'=>'BurkinaFaso','to'=>'Benin','weight'=>1),%('from'=>'BurkinaFaso','to'=>'Ghana','weight'=>1),%('from'=>'BurkinaFaso','to'=>'Togo','weight'=>1),%('from'=>'Guinea','to'=>'IvoryCoast','weight'=>1),%('from'=>'Guinea','to'=>'Senegal','weight'=>1),%('from'=>'Guinea','to'=>'GuineaBissau','weight'=>1),%('from'=>'Guinea','to'=>'Liberia','weight'=>1),%('from'=>'Guinea','to'=>'SierraLeone','weight'=>1),%('from'=>'IvoryCoast','to'=>'Ghana','weight'=>1),%('from'=>'IvoryCoast','to'=>'Liberia','weight'=>1),%('from'=>'Senegal','to'=>'GuineaBissau','weight'=>1),%('from'=>'Senegal','to'=>'Gambia','weight'=>1),%('from'=>'Benin','to'=>'Nigeria','weight'=>1),%('from'=>'Benin','to'=>'Togo','weight'=>1),%('from'=>'Nigeria','to'=>'Cameroon','weight'=>1),%('from'=>'Cameroon','to'=>'CentralAfricanRepublic','weight'=>1),%('from'=>'Cameroon','to'=>'EquatorialGuinea','weight'=>1),%('from'=>'Cameroon','to'=>'Gabon','weight'=>1),%('from'=>'Cameroon','to'=>'RepublicCongo','weight'=>1),%('from'=>'CentralAfricanRepublic','to'=>'SouthSudan','weight'=>1),%('from'=>'CentralAfricanRepublic','to'=>'RepublicCongo','weight'=>1),%('from'=>'CentralAfricanRepublic','to'=>'DemocraticRepublicCongo','weight'=>1),%('from'=>'Eritrea','to'=>'Ethiopia','weight'=>1),%('from'=>'Eritrea','to'=>'SouthSudan','weight'=>1),%('from'=>'Eritrea','to'=>'Djibouti','weight'=>1),%('from'=>'Ethiopia','to'=>'SouthSudan','weight'=>1),%('from'=>'Ethiopia','to'=>'Djibouti','weight'=>1),%('from'=>'Ethiopia','to'=>'Kenya','weight'=>1),%('from'=>'Ethiopia','to'=>'Somalia','weight'=>1),%('from'=>'SouthSudan','to'=>'DemocraticRepublicCongo','weight'=>1),%('from'=>'SouthSudan','to'=>'Kenya','weight'=>1),%('from'=>'SouthSudan','to'=>'Uganda','weight'=>1),%('from'=>'Ghana','to'=>'Togo','weight'=>1),%('from'=>'Liberia','to'=>'SierraLeone','weight'=>1),%('from'=>'EquatorialGuinea','to'=>'Gabon','weight'=>1),%('from'=>'Gabon','to'=>'RepublicCongo','weight'=>1),%('from'=>'RepublicCongo','to'=>'DemocraticRepublicCongo','weight'=>1),%('from'=>'RepublicCongo','to'=>'Angola','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Uganda','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Angola','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Burundi','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Rwanda','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Tanzania','weight'=>1),%('from'=>'DemocraticRepublicCongo','to'=>'Zambia','weight'=>1),%('from'=>'Djibouti','to'=>'Somalia','weight'=>1),%('from'=>'Kenya','to'=>'Somalia','weight'=>1),%('from'=>'Kenya','to'=>'Uganda','weight'=>1),%('from'=>'Kenya','to'=>'Tanzania','weight'=>1),%('from'=>'Uganda','to'=>'Rwanda','weight'=>1),%('from'=>'Uganda','to'=>'Tanzania','weight'=>1),%('from'=>'Angola','to'=>'Zambia','weight'=>1),%('from'=>'Angola','to'=>'Namibia','weight'=>1),%('from'=>'Burundi','to'=>'Rwanda','weight'=>1),%('from'=>'Burundi','to'=>'Tanzania','weight'=>1),%('from'=>'Rwanda','to'=>'Tanzania','weight'=>1),%('from'=>'Tanzania','to'=>'Zambia','weight'=>1),%('from'=>'Tanzania','to'=>'Malawi','weight'=>1),%('from'=>'Tanzania','to'=>'Mozambique','weight'=>1),%('from'=>'Zambia','to'=>'Namibia','weight'=>1),%('from'=>'Zambia','to'=>'Malawi','weight'=>1),%('from'=>'Zambia','to'=>'Mozambique','weight'=>1),%('from'=>'Zambia','to'=>'Botswana','weight'=>1),%('from'=>'Zambia','to'=>'Zimbabwe','weight'=>1),%('from'=>'Namibia','to'=>'Botswana','weight'=>1),%('from'=>'Namibia','to'=>'SouthAfrica','weight'=>1),%('from'=>'Malawi','to'=>'Mozambique','weight'=>1),%('from'=>'Mozambique','to'=>'Zimbabwe','weight'=>1),%('from'=>'Mozambique','to'=>'SouthAfrica','weight'=>1),%('from'=>'Mozambique','to'=>'Swaziland','weight'=>1),%('from'=>'Botswana','to'=>'Zimbabwe','weight'=>1),%('from'=>'Botswana','to'=>'SouthAfrica','weight'=>1),%('from'=>'Zimbabwe','to'=>'SouthAfrica','weight'=>1),%('from'=>'SouthAfrica','to'=>'Swaziland','weight'=>1),%('from'=>'SouthAfrica','to'=>'Lesotho','weight'=>1);

deduce-type(@africaEdges)

Vector(Struct([from, to, weight], [Str, Str, Int]), 109)

In [22]:
sink records-summary(@africaEdges)

+---------------------+-------------------------------+-------------+
| to                  | from                          | weight      |
+---------------------+-------------------------------+-------------+
| Tanzania      => 5  | Algeria                 => 7  | Min    => 1 |
| SouthSudan    => 4  | Mali                    => 6  | 1st-Qu => 1 |
| SouthAfrica   => 4  | DemocraticRepublicCongo => 6  | Mean   => 1 |
| Mozambique    => 3  | Guinea                  => 5  | Median => 1 |
| Senegal       => 3  | Zambia                  => 5  | 3rd-Qu => 1 |
| RepublicCongo => 3  | Libya                   => 5  | Max    => 1 |
| WesternSahara => 3  | Niger                   => 4  |             |
| (Other)       => 84 | (Other)                 => 71 |             |
+---------------------+-------------------------------+-------------+


In [23]:
my %africaCoords = 'Algeria'=>[3.0,28.0],'Libya'=>[17.0,25.0],'Mali'=>[-4.0,17.0],'Mauritania'=>[-12.0,20.0],'Morocco'=>[-5.0,32.0],'Niger'=>[8.0,16.0],'Tunisia'=>[9.0,34.0],'WesternSahara'=>[-13.0,24.5],'Chad'=>[19.0,15.0],'Egypt'=>[30.0,27.0],'Sudan'=>[30.0,13.8],'BurkinaFaso'=>[-2.0,13.0],'Guinea'=>[-10.0,11.0],'IvoryCoast'=>[-5.0,8.0],'Senegal'=>[-14.0,14.0],'Benin'=>[2.25,9.5],'Nigeria'=>[8.0,10.0],'Cameroon'=>[12.0,6.0],'CentralAfricanRepublic'=>[21.0,7.0],'Eritrea'=>[39.0,15.0],'Ethiopia'=>[38.0,8.0],'SouthSudan'=>[30.51,6.7],'Ghana'=>[-2.0,8.0],'Togo'=>[1.1667,8.0],'GuineaBissau'=>[-15.0,12.0],'Liberia'=>[-9.5,6.5],'SierraLeone'=>[-11.5,8.5],'Gambia'=>[-16.5667,13.4667],'EquatorialGuinea'=>[10.0,2.0],'Gabon'=>[11.75,-1.0],'RepublicCongo'=>[15.0,-1.0],'DemocraticRepublicCongo'=>[25.0,0.0],'Djibouti'=>[43.0,11.5],'Kenya'=>[38.0,1.0],'Somalia'=>[49.0,10.0],'Uganda'=>[32.0,1.0],'Angola'=>[18.5,-12.5],'Burundi'=>[30.0,-3.5],'Rwanda'=>[30.0,-2.0],'Tanzania'=>[35.0,-6.0],'Zambia'=>[30.0,-15.0],'Namibia'=>[17.0,-22.0],'Malawi'=>[34.0,-13.5],'Mozambique'=>[35.0,-18.25],'Botswana'=>[24.0,-22.0],'Zimbabwe'=>[30.0,-20.0],'SouthAfrica'=>[24.0,-29.0],'Swaziland'=>[31.5,-26.5],'Lesotho'=>[28.5,-29.5];
%africaCoords .= deepmap(*.Numeric);
deduce-type(%africaCoords)

Assoc(Atom((Str)), Vector(Atom((Rat)), 2), 49)

In [24]:
sink records-summary(%africaCoords.values)

+---------------------+---------------------+
| 1                   | 0                   |
+---------------------+---------------------+
| Min    => -29.5     | Min    => -16.5667  |
| 1st-Qu => -2.75     | 1st-Qu => -2        |
| Mean   => 4.5044224 | Mean   => 15.094082 |
| Median => 8         | Median => 17        |
| 3rd-Qu => 13.9      | 3rd-Qu => 30        |
| Max    => 34        | Max    => 49        |
+---------------------+---------------------+


In [25]:
my $africaGraph = Graph.new(@africaEdges);

Graph(vertexes => 49, edges => 109, directed => False)

In [26]:
my $africanTree = $africaGraph.find-spanning-tree

Graph(vertexes => 49, edges => 48, directed => False)

In [27]:
#% js
js-d3-graph-plot(
    $africaGraph.edges,
    highlight => {Blue => [|$africanTree.vertex-list, |$africanTree.edges]},
    vertex-coordinates => %africaCoords,
    width => 600,
    height => 600,
    vertex-label-color => 'none',
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    )

In [49]:
#% html
$africaGraph.vertex-coordinates = %africaCoords;
$africaGraph.dot(
    highlight => {Blue => [|$africanTree.vertex-list, |$africanTree.edges]},
    graph-size => (6, 6),
    edge-thickness => 20,
    node-width => 2,
    :!node-labels,
    engine => 'neato'
    ):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Egypt -->
 
 Egypt 
 
 
<!-- Libya -->
 
 Libya 
 
 
<!-- Egypt--Libya -->
 
 Egypt--Libya 
 
 
<!-- Sudan -->
 
 Sudan 
 
 
<!-- Egypt--Sudan -->
 
 Egypt--Sudan 
 
 
<!-- Guinea -->
 
 Guinea 
 
 
<!-- Senegal -->
 
 Senegal 
 
 
<!-- Guinea--Senegal -->
 
 Guinea--Senegal 
 
 
<!-- SierraLeone -->
 
 SierraLeone 
 
 
<!-- Guinea--SierraLeone -->
 
 Guinea--SierraLeone 
 
 
<!-- IvoryCoast -->
 
 IvoryCoast 
 
 
<!-- Guinea--IvoryCoast -->
 
 Guinea--IvoryCoast 
 
 
<!-- GuineaBissau -->
 
 GuineaBissau 
 
 
<!-- Guinea--GuineaBissau -->
 
 Guinea--GuineaBissau 
 
 
<!-- Rwanda -->
 
 Rwanda 
 
 
<!-- Burundi -->
 
 Burundi 
 
 
<!-- Rwanda--Burundi -->
 
 Rwanda--Burundi 
 
 
<!-- Gambia -->
 
 Gambia 
 
 
<!-- Botswana -->
 
 Botswana 
 
 
<!-- SouthAfrica -->
 
 SouthAfrica 
 
 
<!-- Botswana--SouthAfrica -->
 
 Botswana--SouthAfrica 
 
 
<!-- Namibia -->
 
 Namibia 
 
 
<!-- Botswana--Namibia -->
 
 Botswana--Namibia 
 
 
<!-- Zimbabwe -->
 
 Zimbabwe 
 
 
<!-- Botswana--Zimbabwe -->
 
 Botswana--Zimbabwe 
 
 
<!-- RepublicCongo -->
 
 RepublicCongo 
 
 
<!-- Liberia -->
 
 Liberia 
 
 
<!-- Liberia--Guinea -->
 
 Liberia--Guinea 
 
 
<!-- Senegal--Gambia -->
 
 Senegal--Gambia 
 
 
<!-- Senegal--GuineaBissau -->
 
 Senegal--GuineaBissau 
 
 
<!-- SouthSudan -->
 
 SouthSudan 
 
 
<!-- Kenya -->
 
 Kenya 
 
 
<!-- SouthSudan--Kenya -->
 
 SouthSudan--Kenya 
 
 
<!-- CentralAfricanRepublic -->
 
 CentralAfricanRepublic 
 
 
<!-- SouthSudan--CentralAfricanRepublic -->
 
 SouthSudan--CentralAfricanRepublic 
 
 
<!-- Uganda -->
 
 Uganda 
 
 
<!-- SouthSudan--Uganda -->
 
 SouthSudan--Uganda 
 
 
<!-- Eritrea -->
 
 Eritrea 
 
 
<!-- SouthSudan--Eritrea -->
 
 SouthSudan--Eritrea 
 
 
<!-- SierraLeone--Liberia -->
 
 SierraLeone--Liberia 
 
 
<!-- IvoryCoast--Liberia -->
 
 IvoryCoast--Liberia 
 
 
<!-- Ghana -->
 
 Ghana 
 
 
<!-- IvoryCoast--Ghana -->
 
 IvoryCoast--Ghana 
 
 
<!-- Swaziland -->
 
 Swaziland 
 
 
<!-- Swaziland--SouthAfrica -->
 
 Swaziland--SouthAfrica 
 
 
<!-- Mozambique -->
 
 Mozambique 
 
 
<!-- Swaziland--Mozambique -->
 
 Swaziland--Mozambique 
 
 
<!-- Gabon -->
 
 Gabon 
 
 
<!-- Gabon--RepublicCongo -->
 
 Gabon--RepublicCongo 
 
 
<!-- Tanzania -->
 
 Tanzania 
 
 
<!-- Tanzania--Rwanda -->
 
 Tanzania--Rwanda 
 
 
<!-- Tanzania--Kenya -->
 
 Tanzania--Kenya 
 
 
<!-- DemocraticRepublicCongo -->
 
 DemocraticRepublicCongo 
 
 
<!-- Tanzania--DemocraticRepublicCongo -->
 
 Tanzania--DemocraticRepublicCongo 
 
 
<!-- Tanzania--Uganda -->
 
 Tanzania--Uganda 
 
 
<!-- Tanzania--Burundi -->
 
 Tanzania--Burundi 
 
 
<!-- Lesotho -->
 
 Lesotho 
 
 
<!-- Nigeria -->
 
 Nigeria 
 
 
<!-- Benin -->
 
 Benin 
 
 
<!-- Nigeria--Benin -->
 
 Nigeria--Benin 
 
 
<!-- Niger -->
 
 Niger 
 
 
<!-- Nigeria--Niger -->
 
 Nigeria--Niger 
 
 
<!-- Cameroon -->
 
 Cameroon 
 
 
<!-- Nigeria--Cameroon -->
 
 Nigeria--Cameroon 
 
 
<!-- Mauritania -->
 
 Mauritania 
 
 
<!-- Mauritania--Senegal -->
 
 Mauritania--Senegal 
 
 
<!-- Algeria -->
 
 Algeria 
 
 
<!-- Mauritania--Algeria -->
 
 Mauritania--Algeria 
 
 
<!-- DemocraticRepublicCongo--Rwanda -->
 
 DemocraticRepublicCongo--Rwanda 
 
 
<!-- DemocraticRepublicCongo--RepublicCongo -->
 
 DemocraticRepublicCongo--RepublicCongo 
 
 
<!-- DemocraticRepublicCongo--SouthSudan -->
 
 DemocraticRepublicCongo--SouthSudan 
 
 
<!-- DemocraticRepublicCongo--CentralAfricanRepublic -->
 
 DemocraticRepublicCongo--CentralAfricanRepublic 
 
 
<!-- DemocraticRepublicCongo--Uganda -->
 
 DemocraticRepublicCongo--Uganda 
 
 
<!-- DemocraticRepublicCongo--Burundi -->
 
 DemocraticRepublicCongo--Burundi 
 
 
<!-- CentralAfricanRepublic--RepublicCongo -->
 
 CentralAfricanRepublic--RepublicCongo 
 
 
<!-- Zambia -->
 
 Zambia 
 
 
<!--

-------

## Graph by list-line-plot

In [28]:
my $k = 0;
my @africaEdges2 = @africaEdges.map({ 
    [
        %(x => %africaCoords{$_<from>}.head, y => %africaCoords{$_<from>}.tail, group => ~$k), 
        %(x => %africaCoords{$_<to>}.head, y => %africaCoords{$_<to>}.tail, group => ~$k++), 
    ]}).flat

[{group => 0, x => 3, y => 28} {group => 0, x => 17, y => 25} {group => 1, x => 3, y => 28} {group => 1, x => -4, y => 17} {group => 2, x => 3, y => 28} {group => 2, x => -12, y => 20} {group => 3, x => 3, y => 28} {group => 3, x => -5, y => 32} {group => 4, x => 3, y => 28} {group => 4, x => 8, y => 16} {group => 5, x => 3, y => 28} {group => 5, x => 9, y => 34} {group => 6, x => 3, y => 28} {group => 6, x => -13, y => 24.5} {group => 7, x => 17, y => 25} {group => 7, x => 8, y => 16} {group => 8, x => 17, y => 25} {group => 8, x => 9, y => 34} {group => 9, x => 17, y => 25} {group => 9, x => 19, y => 15} {group => 10, x => 17, y => 25} {group => 10, x => 30, y => 27} {group => 11, x => 17, y => 25} {group => 11, x => 30, y => 13.8} {group => 12, x => -4, y => 17} {group => 12, x => -12, y => 20} {group => 13, x => -4, y => 17} {group => 13, x => 8, y => 16} {group => 14, x => -4, y => 17} {group => 14, x => -2, y => 13} {group => 15, x => -4, y => 17} {group => 15, x => -10, y => 11}

In [29]:
#%js
js-d3-list-line-plot(@africaEdges2,
    width => 600,
    height => 600,
    :grid-lines,
    :$background, 
    color-scheme => 'schemePaired',
    stroke-width => 3,
    :!legends
).subst('.range(d3.schemePaired);', '.range(["SteelBlue"]);')